#### exercise 1

In [1]:
# import libs
import numpy as np
import polars as pl
import pandas as pd

In [2]:
# read csv to dataframe
pl_ames = pl.read_csv("../AmesHousing.csv")
pl_xlsx = pl.read_excel("../Neighborhood_names.xlsx")

In [3]:
pl_ames.describe()

statistic,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,…,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
str,f64,f64,f64,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,str,str,str,str,str,f64,str,str,str,str,str,str,str,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,str,str,f64,str,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,str,str,f64
"""count""",2930.0,2930.0,2930.0,"""2930""",2440.0,2930.0,"""2930""","""2930""","""2930""","""2930""","""2930""","""2930""","""2930""","""2930""","""2930""","""2930""","""2930""","""2930""",2930.0,2930.0,2930.0,2930.0,"""2930""","""2930""","""2930""","""2930""","""2907""",2907.0,"""2930""","""2930""","""2930""","""2929""","""2929""","""2926""","""2929""",2929.0,…,2930.0,2930.0,2930.0,2928.0,2928.0,2930.0,2930.0,2930.0,2930.0,"""2930""",2930.0,"""2930""",2930.0,"""2930""","""2930""",2771.0,"""2928""",2929.0,2929.0,"""2929""","""2929""","""2930""",2930.0,2930.0,2930.0,2930.0,2930.0,2930.0,"""2930""","""2930""","""2930""",2930.0,2930.0,2930.0,"""2930""","""2930""",2930.0
"""null_count""",0.0,0.0,0.0,"""0""",490.0,0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,"""0""","""0""","""0""","""0""","""23""",23.0,"""0""","""0""","""0""","""1""","""1""","""4""","""1""",1.0,…,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,"""0""",0.0,"""0""",0.0,"""0""","""0""",159.0,"""2""",1.0,1.0,"""1""","""1""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,"""0""","""0""","""0""",0.0,0.0,0.0,"""0""","""0""",0.0
"""mean""",1465.5,7.1446e8,57.387372,null,69.22459,10147.921843,null,null,null,null,null,null,null,null,null,null,null,null,6.094881,5.56314,1971.356314,1984.266553,null,null,null,null,null,101.896801,null,null,null,null,null,null,null,442.629566,…,335.455973,4.676792,1499.690444,0.431352,0.061134,1.566553,0.379522,2.854266,1.044369,null,6.443003,null,0.599317,null,null,1978.132443,null,1.766815,472.819734,null,null,null,93.751877,47.533447,23.011604,2.592491,16.002048,2.243345,null,null,null,50.635154,6.216041,2007.790444,null,null,180796.060068
"""std""",845.96247,1.8873e8,42.638025,null,23.365335,7880.017759,null,null,null,null,null,null,null,null,null,null,null,null,1.411026,1.111537,30.245361,20.860286,null,null,null,null,null,179.112611,null,null,null,null,null,null,null,455.590839,…,428.395715,46.31051,505.508887,0.52482,0.245254,0.552941,0.502629,0.827731,0.214076,null,1.572964,null,0.647921,null,null,25.528411,null,0.760566,215.046549,null,null,null,126.361562,67.4834,64.139059,25.141331,56.08737,35.597181,null,null,null,566.344288,2.714492,1.316613,null,null,79886.692357
"""min""",1.0,5.263011e8,20.0,"""A (agr)""",21.0,1300.0,"""Grvl""","""Grvl""","""IR1""","""Bnk""","""AllPub""","""Corner""","""Gtl""","""Blmngtn""","""Artery""","""Artery""","""1Fam""","""1.5Fin""",1.0,1.0,1872.0,1950.0,"""Flat""","""ClyTile""","""AsbShng""","""AsbShng""","""BrkCmn""",0.0,"""Ex""","""Ex""","""BrkTil""","""Ex""","""Ex""","""Av""","""ALQ""",0.0,…,0.0,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Ex""",2.0,"""Maj1""",0.0,"""Ex""","""2Types""",1895.0,"""Fin""",0.0,0.0,"""Ex""","""Ex""","""N""",0.0,0.0,0.0,0.0,0.0,0.0,"""Ex""","""GdPrv""","""Elev""",0.0,1.0,2006.0,"""COD""","""Abnorml""",12789

#### exercise 2

In [4]:
# split dataframe into subsets by type
pl_ames_int = pl_ames.select(pl.col(pl.Int64))
pl_ames_string = pl_ames.select(pl.col(pl.String))

In [5]:
# show all distinct types
print(set(pl_ames.dtypes))

# count number of each type
number_int = pl_ames.dtypes.count(pl.Int64)
number_string = pl_ames.dtypes.count(pl.String)

print(f"Number of int columns: {number_int}, Number of string columns: {number_string}")

{String, Int64}
Number of int columns: 39, Number of string columns: 43


In [7]:
# find all columns with missing values, and show how many
pl_ames.null_count().transpose(include_header=True).filter(pl.col("column_0") > 0).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Lot Frontage""","""Mas Vnr Type""","""Mas Vnr Area""","""Bsmt Qual""","""Bsmt Cond""","""Bsmt Exposure""","""BsmtFin Type 1""","""BsmtFin SF 1""","""BsmtFin Type 2""","""BsmtFin SF 2""","""Bsmt Unf SF""","""Total Bsmt SF""","""Electrical""","""Bsmt Full Bath""","""Bsmt Half Bath""","""Garage Yr Blt""","""Garage Finish""","""Garage Cars""","""Garage Area""","""Garage Qual""","""Garage Cond"""
"""490""","""23""","""23""","""1""","""1""","""4""","""1""","""1""","""2""","""1""","""1""","""1""","""1""","""2""","""2""","""159""","""2""","""1""","""1""","""1""","""1"""


In [8]:
# is sales price complete
pl_ames["SalePrice"].is_null().any()

False

In [9]:
pd_ames = pd.read_csv("../AmesHousing.csv")
pd_xlsx = pd.read_excel("../Neighborhood_names.xlsx")

In [ ]:
pd_ames.isnull().sum()

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Frontage      490
                 ... 
Mo Sold             0
Yr Sold             0
Sale Type           0
Sale Condition      0
SalePrice           0
Length: 82, dtype: int64

In [24]:
# construct summary statistics
pl_ames_int.describe()

statistic,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",2930.0,2930.0,2930.0,2440.0,2930.0,2930.0,2930.0,2930.0,2930.0,2907.0,2929.0,2929.0,2929.0,2929.0,2930.0,2930.0,2930.0,2930.0,2928.0,2928.0,2930.0,2930.0,2930.0,2930.0,2930.0,2930.0,2771.0,2929.0,2929.0,2930.0,2930.0,2930.0,2930.0,2930.0,2930.0,2930.0,2930.0,2930.0,2930.0
"""null_count""",0.0,0.0,0.0,490.0,0.0,0.0,0.0,0.0,0.0,23.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,159.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",1465.5,7.1446e8,57.387372,69.22459,10147.921843,6.094881,5.56314,1971.356314,1984.266553,101.896801,442.629566,49.722431,559.262547,1051.614544,1159.557679,335.455973,4.676792,1499.690444,0.431352,0.061134,1.566553,0.379522,2.854266,1.044369,6.443003,0.599317,1978.132443,1.766815,472.819734,93.751877,47.533447,23.011604,2.592491,16.002048,2.243345,50.635154,6.216041,2007.790444,180796.060068
"""std""",845.96247,1.8873e8,42.638025,23.365335,7880.017759,1.411026,1.111537,30.245361,20.860286,179.112611,455.590839,169.168476,439.494153,440.615067,391.890885,428.395715,46.31051,505.508887,0.52482,0.245254,0.552941,0.502629,0.827731,0.214076,1.572964,0.647921,25.528411,0.760566,215.046549,126.361562,67.4834,64.139059,25.141331,56.08737,35.597181,566.344288,2.714492,1.316613,79886.692357
"""min""",1.0,5.263011e8,20.0,21.0,1300.0,1.0,1.0,1872.0,1950.0,0.0,0.0,0.0,0.0,0.0,334.0,0.0,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1895.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2006.0,12789.0
"""25%""",733.0,5.2847702e8,20.0,58.0,7440.0,5.0,5.0,1954.0,1965.0,0.0,0.0,0.0,219.0,793.0,876.0,0.0,0.0,1126.0,0.0,0.0,1.0,0.0,2.0,1.0,5.0,0.0,1960.0,1.0,320.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2007.0,129500.0
"""50%""",1466.0,5.3545403e8,50.0,68.0,9439.0,6.0,5.0,1973.0,1993.0,0.0,370.0,0.0,466.0,990.0,1084.0,0.0,0.0,1442.0,0.0,0.0,2.0,0.0,3.0,1.0,6.0,1.0,1979.0,2.0,480.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,6.0,2008.0,160000.0
"""75%""",2198.0,9.071811e8,70.0,80.0,11556.0,7.0,6.0,2001.0,2004.0,164.0,734.0,0.0,802.0,1302.0,1384.0,704.0,0.0,1743.0,1.0,0.0,2.0,1.0,3.0,1.0,7.0,1.0,2002.0,2.0,576.0,168.0,70.0,0.0,0.0,0.0,0.0,0.0,8.0,2009.0,213500.0
"""max""",2930.0,1.0071e9,190.0,313.0,215245.0,10.0,9.0,2010.0,2010.0,1600.0,5644.0,1526.0,2336.0,6110.0,5095.0,2065.0,1064.0,5642.0,3.0,2.0,4.0,2.0,8.0,3.0,15.0,4.0,2207.0,5.0,1488.0,1424.0,742.0,1012.0,508.0,576.0,800.0,17000.0,12.0,2010.0,755000.0
